In [ ]:
!wget -nc -q https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat
!wget -nc -q https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat

In [ ]:
import networkx as nx
import math
from PIL import Image, ImageDraw
import pyproj
import numpy as np
import matplotlib.pyplot as plt
import geopy.distance
import pandas as pd

In [ ]:
G = nx.DiGraph()

In [ ]:
airportsDF = pd.read_csv('airports.dat', header=None, names=['id', 'name', 'city', 'country', 'IATA', 'ICAO', 'lat', 'lon', 'alt', 'timezone', 'DST', 'Tz', 'type', 'source'])

for index, row in airportsDF.iterrows():
    if row['IATA'] == '\\N':
        #print(row['IATA'])
        continue
    G.add_node(row['IATA'], pos=(row['lat'],row['lon']))

In [ ]:
print(len(G.nodes))
print(len(G.edges))

In [ ]:
print(G.nodes['TRF'])
#print(G.nodes['AOS'])

In [ ]:
routesDF = pd.read_csv('routes.dat', header=None, names=['airline', 'airlineID', 'source', 'sourceID', 'dest', 'destID', 'codeshare', 'stops', 'equipment'])

for index, row in routesDF.iterrows():
    if(row['source'] not in G.nodes or row['dest'] not in G.nodes):
        continue
    dist = geopy.distance.distance((G.nodes[row['source']]['pos']), (G.nodes[row['dest']]['pos'])).km
    G.add_edge(row['source'], row['dest'], weight=dist)

In [ ]:
print(len(G.nodes))

In [ ]:
# remove nodes with zero degree
#G.remove_nodes_from(list(nx.isolates(G)))

In [ ]:
print(len(G.nodes))
print(len(G.edges))

In [ ]:
G.is_directed()

In [ ]:
G.remove_nodes_from(list(nx.isolates(G)))
H = G.copy()
i=0
while i == 0:
  i = 1
  H = G.copy()
  for node in H.in_degree:
    if node[1] == 0:
      i = 0
      G.remove_node(node[0])
  H = G.copy()
  for node in H.out_degree:
    if node[1] == 0:
      i = 0
      G.remove_node(node[0])

In [ ]:
nodes = sorted(G.degree, key=lambda x: x[1])
print(nodes)
print(G.degree)

In [ ]:
print(len(G.nodes))

In [ ]:
nodes = sorted(G.degree, key=lambda x: x[1])
print(nodes)

In [ ]:
print(len(G.nodes))
print(len(G.edges))

In [ ]:
print(G.degree("LIS"))
print(G.degree("ZYL"))
print(G.degree("AAE"))

In [ ]:
# Graph density
nx.density(G)

In [ ]:
# Graph diameter
# nx.diameter(G)

In [ ]:
# Degree Distribution
degrees = [G.degree(n) for n in G.nodes()]
print(max(degrees))
plt.hist(degrees, bins=np.logspace(np.log10(2),np.log10(max(degrees)), 20))
plt.xscale("log")
# for i in range(0, max(degrees)):
#   print(i, degrees.count(i))


In [ ]:
import collections

degrees = [G.degree(n) for n in G.nodes()]
degree_sequence = sorted(degrees, reverse=True) # degree sequence
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())
cs = np.cumsum(cnt)
plt.loglog(deg, cs, 'bo')
plt.title("Cumulative Distribution plot")
plt.ylabel("Sample with value > Degree")
plt.xlabel("Degree")
plt.show()

In [ ]:
# compute graph degree centrality
dc = nx.degree_centrality(G)

max_dc = max(dc, key=dc.get)
print(max_dc)
print(dc[max_dc])

min_dc = min(dc, key=dc.get)
print(min_dc)
print(dc[min_dc])

In [ ]:
# compute eigen vector centrality
ec = nx.eigenvector_centrality(G)

max_ec = max(ec, key=ec.get)
print(max_ec)
print(ec[max_ec])

min_ec = min(ec, key=ec.get)
print(min_ec)
print(ec[min_ec])

In [ ]:
# compute closeness centrality
cc = nx.closeness_centrality(G)

max_cc = max(cc, key=cc.get)
print(max_cc)
print(cc[max_cc])

min_cc = min(cc, key=cc.get)
print(min_cc)
print(cc[min_cc])

In [ ]:
# compute betweenness centrality
bc = nx.betweenness_centrality(G)

max_bc = max(bc, key=bc.get)
print(max_bc)
print(bc[max_bc])

min_bc = min(bc, key=bc.get)
print(min_bc)
print(bc[min_bc])

In [ ]:
# compute pagerank
pr = nx.pagerank(G)

max_pr = max(pr, key=pr.get)
print(max_pr)
print(pr[max_pr])

min_pr = min(pr, key=pr.get)
print(min_pr)
print(pr[min_pr])

In [ ]:
# compute clustering coefficient
cl = nx.clustering(G)

max_cl = max(cl, key=cl.get)
print(max_cl)
print(cl[max_cl])

min_cl = min(cl, key=cl.get)
print(min_cl)
print(cl[min_cl])

In [ ]:
# generate strongly connected components
sccs = nx.strongly_connected_components(G)
print([len(Gc) for Gc in sccs])

sccs = nx.strongly_connected_components(G)
sccs_graphs = (G.subgraph(c) for c in sccs)

largest_scc = list(sccs_graphs)[0]
print(len(largest_scc))
print(largest_scc)
# for node in largest_scc:
#   print(G.nodes[node].get('country'))

In [ ]:
# average shortest path
#TODO: add weights
avg_path_length = nx.average_shortest_path_length(largest_scc)
print(avg_path_length)

In [ ]:
nx.diameter(largest_scc)

In [ ]:
G.nodes['LIS']

In [ ]:
# Draw the graph in a map
# latitude = float(G.nodes['LIS'].get('pos')[0])
# longitude = float(G.nodes['LIS'].get('pos')[1])

# #resolution of image you are using
# mapWidth = 2048;
# mapHeight = 1025;

# #get x value
# x = (mapWidth*(180+longitude)/360)%mapWidth+(mapWidth/2);
# #convert from degrees to radians
# latRad = latitude*math.pi/180;
# #get y value
# mercN = math.log(math.tan((math.pi/4)+(latRad/2)));
# y = (mapHeight/2)-(mapWidth*mercN/(2*math.pi));
# print(x, y)

# Define the source and destination projections
source_projection = pyproj.Proj(init='epsg:4326')  # WGS 84 (standard lat/lon)
destination_projection = pyproj.Proj(init='epsg:3857')  # Web Mercator

# Input geographic coordinates (longitude, latitude)
lon, lat = float(G.nodes['LIS'].get('pos')[0]), float(G.nodes['LIS'].get('pos')[1])  # Example coordinates for New York City

# Project the coordinates to the destination projection
x, y = pyproj.transform(source_projection, destination_projection, lon, lat)
print(x, y)

def project_to_pixel(x, y, scale, x_offset, y_offset):
    pixel_x = (x - x_offset) * scale
    pixel_y = (y - y_offset) * scale
    return pixel_x, pixel_y

# Calculate the pixel coordinates
scale = 0.00001 # Adjust as needed
x_offset, y_offset = 0, 0  # Adjust as needed
x, y = project_to_pixel(x, y, scale, x_offset, y_offset)
print(x, y)
image = Image.open("openflights-apdb-2048.png")
draw = ImageDraw.Draw(image)
draw.ellipse((x,y, x+50, y+50), fill="black")
image.show()

In [ ]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

geometry = [Point(xy) for xy in zip(airportsDF['lon'], airportsDF['lat'])]
gdf = GeoDataFrame(airportsDF, geometry=geometry)

# lis = airportsDF.loc[airportsDF['IATA'] == 'LIS']
# geometry = [Point(lis['lon'], lis['lat'])]
# gdf = GeoDataFrame(lis, geometry=geometry)

# #this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(20, 12)), marker='o', markersize=8, color='red')
#gdf.plot(ax=world.plot(figsize=(20, 12)))
#gdf.plot(ax=world.plot(figsize=(20, 12)), marker='o', markersize=8, c=dc[gdf['IATA'].item()], cmap='YlOrRd')
